## 1. Setup

In [ ]:
import os
import glob
import re
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import cv2
from sklearn.metrics import roc_auc_score
from sklearn import model_selection
from torch.optim.lr_scheduler import CosineAnnealingLR

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(38)

In [ ]:
# local notebook settings
# from efficientnet_pytorch_3d import EfficientNet3D
# train_path = 'train_labels.csv'
# test_path = 'sample_submission_csv'
# png_path = 'archive'
# from volumentations import *


# kaggle settings
import sys
sys.path.append('../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D')
from efficientnet_pytorch_3d import EfficientNet3D

sys.path.append('../input/3d-augmentation/volumentations-master')
from volumentations import *

sys.path.append('../input/gradualwarmupschedulerv2')
from warmup_scheduler import GradualWarmupScheduler

train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv'
test_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv'
png_path = '../input/rsna-miccai-png'

In [ ]:
img_size = 256
num_img = 64
mri_types = ["FLAIR", "T1w", "T1wCE", "T2w"]

## 2. Dataloader

In [ ]:
def load_image(scan_id, mri_type, split='train', path=png_path):
    mri = sorted(glob.glob(f"{path}/{split}/{scan_id}/{mri_type}/*.png"), 
                 key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    
    if len(mri) == 0:
        i_img = np.zeros((img_size, img_size, num_img))
    elif len(mri) < num_img:
        i_img = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size))
                        for path in mri[0:num_img]]).T 
        num_zero = num_img - i_img.shape[-1]
        i_img.shape[-1]
        i_img = np.concatenate((i_img, np.zeros((img_size, img_size, num_zero))), -1)
    else: 
        i_img = np.array([cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (img_size, img_size)) 
                         for path in mri[len(mri) // 2 - num_img // 2: len(mri) // 2 + num_img // 2]]).T
            
    return i_img

In [ ]:
class RSNADataset(Dataset):
    def __init__(self, csv, mri_type, mode='train', transform=None):
        self.csv = csv
        self.mode = mode
        self.transform = transform
        self.mri_type = mri_type
        
    def __len__(self): 
        return self.csv.shape[0]
    
    def __getitem__(self, idx): 
        data = self.csv.iloc[idx]
        brat = str(int(data["BraTS21ID"])).zfill(5)
        mgmt = data['MGMT_value']
        
        image = load_image(brat, self.mri_type, self.mode)

        if self.transform is not None:
            data = {'image': image}
            aug_data = self.transform(**data)
            image = aug_data['image']
            
        image = image.transpose(2,1,0)           
        image = image / 255
        
        if self.mode == 'train': 
            return torch.tensor(image, dtype=torch.float32), torch.tensor(mgmt, dtype=torch.float)
        else: 
            return torch.tensor(image, dtype=torch.float32), brat
            

In [ ]:
transforms_train = Compose([
        Flip(-3, p=0.5),
        GaussianNoise(var_limit=(1, 10), mean=6, p=0.35),
        ElasticTransformPseudo2D(alpha=300, sigma=300 * 0.05, alpha_affine=300 * 0.03, p=0.1),
        RandomScale(scale_limit=[1, 1.2], p=1),
        CenterCrop((img_size, img_size, num_img), p=1)
    ], p=1)

In [ ]:
# # for testing the cell below

# train_df = pd.read_csv(train_path)
# df_train, df_val = model_selection.train_test_split(train_df, test_size=0.2, random_state=38, stratify=train_df["MGMT_value"])
# train_dataset = RSNADataset(df_train, mri_types[0], transform=transforms_train)

In [ ]:
# # creates animation to check the dataset

# from matplotlib import animation, rc
# rc('animation', html='jshtml')


# def create_animation(ims):
#     fig = plt.figure(figsize=(6, 6))
#     plt.axis('off')
#     im = plt.imshow(ims[0], cmap="gray")

#     def animate_func(i):
#         im.set_array(ims[i])
#         return [im]

#     return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24)

# test = train_dataset.__getitem__(100)
# create_animation(test[0])

## 3.Model

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)
        n_features = self.net._fc.in_features
        self.net._fc = nn.Linear(in_features=n_features, out_features=1, bias=True)
        
    def forward(self, x):
        return self.net(x)

## 4. Train 

In [ ]:
class Trainer:
    def __init__(self, mri_type, model, criterion, optimizer, scheduler_warmup, device):
        self.mri_type = mri_type
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler_warmup = scheduler_warmup
        self.device = device
        self.model_train_losses = []
        self.model_val_losses = []
        self.model_val_auc = []
        
    def fit(self, train_loader, val_loader, n_epochs):
        best_val = 10000

        for epoch in range(n_epochs): 
#             self.scheduler_warmup.step(epoch)
            
            # train
            avg_train = self.train_epoch(train_loader)
            self.model_train_losses.append(avg_train)
            
            print(f'epoch {epoch + 1} train: {avg_train}')
                
            # val
            avg_val, auc = self.val_epoch(val_loader)
            
            self.model_val_losses.append(avg_val)
            self.model_val_auc.append(auc)
            print(f'epoch {epoch + 1} val: {avg_val}, auc: {auc}')

            if avg_val < best_val: 
                print('save model ...')
                best_val = avg_val
                last_model =  f'{self.mri_type}-e{epoch + 1}-loss{avg_val}-auc{auc}.pt'
                torch.save(self.model.state_dict(), f'{self.mri_type}-e{epoch + 1}-loss{avg_val}-auc{auc}.pt')

            print('\n\n')
            
        return last_model
                    
    def train_epoch(self, train_loader):
        train_loss = []
        self.model.train()

        for i, data in tqdm(enumerate(train_loader, 0)): 
            x, y = data
            x = torch.unsqueeze(x, dim=1)
            x, y = x.to(device), y.to(device)
            self.optimizer.zero_grad()

            outputs = self.model(x).squeeze(1)
            loss = self.criterion(outputs, y)
            loss.backward()
            self.optimizer.step()

            train_loss.append(loss.detach().item())
        avg_train = sum(train_loss) / len(train_loss)
        
        return avg_train

        
    def val_epoch(self, val_loader):
        self.model.eval()
        outputs_all = []
        y_all = []
        val_loss = []    

        for i, data in tqdm(enumerate(val_loader, 0)): 
            with torch.no_grad():
                x, y = data 
                x = torch.unsqueeze(x, dim=1)
                x, y = x.to(device), y.to(device)

                outputs = self.model(x).squeeze(1)
                loss = self.criterion(outputs, y)
                
                outputs = torch.sigmoid(outputs)
                outputs_all.extend(outputs.tolist())
                y_all.extend(y.tolist())
                val_loss.append(loss.detach().item())      
        avg_val = sum(val_loss) / len(val_loss)
        auc = roc_auc_score(y_all, outputs_all)
        
        return avg_val, auc

In [ ]:
class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

In [ ]:
train_df = pd.read_csv(train_path)
df_train, df_val = model_selection.train_test_split(train_df, test_size=0.2, random_state=38, stratify=train_df["MGMT_value"])

In [ ]:
cosine_epo = 15
warmup_epo = 2
n_epochs = cosine_epo + warmup_epo

models = []

for mri_type in mri_types: 
    model = Model()
    criterion = functional.binary_cross_entropy_with_logits
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
#     optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)

#     scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, cosine_epo)
#     scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

    device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    train_dataset = RSNADataset(df_train, mri_type, transform=transforms_train)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=4, num_workers=4)

    val_dataset = RSNADataset(df_val, mri_type)
    val_loader = DataLoader(val_dataset, shuffle=False, batch_size=4, num_workers=4)

#     mri_trainer = Trainer(mri_type, model, criterion, optimizer, scheduler_warmup, device)
    scheduler_warmup=None
    mri_trainer = Trainer(mri_type, model, criterion, optimizer, scheduler_warmup, device)

    last_best_model = mri_trainer.fit(train_loader, val_loader, n_epochs)
    
    models.append(last_best_model)

## 5. Predict

In [ ]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')

df_test = pd.read_csv(test_path)

write = 0

for trained_model in range(len(models)): 
    model = Model()
    model.to(device)
    checkpoint = torch.load(models[trained_model])
    model.load_state_dict(checkpoint)
    model.eval()
    
    test_dataset = RSNADataset(df_test, mri_type=mri_types[trained_model], mode="test")
    test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=8, pin_memory=True)
    
    y_pred = []

    for e, batch in enumerate(test_loader):
        print(f"{e + 1}/{len(test_loader)}", end="\r")
        
        batch, brat = batch
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch.to(device).unsqueeze(1))).squeeze()
            y_pred.extend(tmp_pred.tolist())
    
    y_pred = np.array(y_pred)
    
    if write == 0:
        write = 1
        submission = pd.DataFrame({"BraTS21ID": df_test['BraTS21ID'].apply(lambda x: str(x).zfill(5)), "MGMT_value": y_pred})
    else: 
        submission['MGMT_value'] += y_pred

submission['MGMT_value'] /= len(models)

In [ ]:
submission.to_csv("submission.csv", index=False)

In [ ]:
# try to take off data augmentation and use 15 epochs instead
# also take off learning rate scheduled, try 3 value: 3e-5, 0.001, 0.0001

# version nay bo scheduler warmup nhung ma keep data augmentation
# chi train duoc 2 epoch thoi, xong no se bi full data loader